In [195]:
# initialization
import urllib.request as ul
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import datetime
import pandas as pd
import json
import csv
import scipy.spatial.distance as ssd
import pickle

#Stripper dependencies
import sys 
sys.modules["regex"] = __import__("re")
import nltk
nltk.data.path.append(".")
try:
    zyx = nltk.data.find('averaged_perceptron_tagger')
    wrt = nltk.data.find('tokenizers/punkt')
except:
    nltk.download("punkt", download_dir=".")
    nltk.download("averaged_perceptron_tagger", download_dir=".")
from collections import Counter
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
#---

abbr = pd.read_csv('abb.csv', header=None, index_col=0, squeeze=True).to_dict()

In [10]:
#accesses the wiki
housewiki = 'https://en.wikipedia.org/wiki/List_of_current_members_of_the_United_States_House_of_Representatives'
html = requests.get(housewiki)
soup = BeautifulSoup(html.text)
housetable = soup.findAll('table',class_="sortable")[2]

#pulls headers from table and creates dictionary
headers = [el.text[:-1] for el in housetable.findAll('th')]
headers[-1] = headers[-1][:-3]
housedict = {}
for el in headers:
    housedict[el] = []

#extracts columns
housetable = housetable.find('tbody').findAll('tr')[1:]
for x in housetable:
    if len(x.findChildren('td'))==9:
        core = x.findChildren('td')
        core.pop(2)
        for el in range(8):
            housedict[headers[el]].append(core[el].text)

#tidying
for e in range(8):
    for x, item in enumerate(housedict[headers[e]]):
        if e==1:
            housedict[headers[e]][x] = housedict[headers[e]][x].replace('\xa0',' ').replace('\n','').strip()
        else:
            housedict[headers[e]][x] = housedict[headers[e]][x].replace('\xa0',' ').replace('\n','')
for el, it in enumerate(housedict['Residence']):
    if housedict['Residence'][el][-3] == '[':
        housedict['Residence'][el] = housedict['Residence'][el][:-3]
    elif housedict['Residence'][el][-4] == '[':
        housedict['Residence'][el] = housedict['Residence'][el][:-4]

for el, it in enumerate(housedict['Born']):
    housedict['Born'][el] = housedict['Born'][el][2:12]

#sends to pandas
df = pd.DataFrame(housedict)

# age calc
df['Born'] = pd.to_datetime(df['Born'], format='%Y-%m-%d')

def dd(df,c,x):
    b=df[c][x].year
    n=pd.Timestamp.now().year
    return n-b
df['Age'] = [dd(df,'Born',x) for x in range(len(df))]

In [183]:
#takes the url tails for each senator
url = 'https://justfacts.votesmart.org/candidate/public-statements'
search = 'https://votesmart.org/search?q='
allrep ='https://votesmart.org/officials/NA/C/-congressional?chamber=L'
html = requests.get(allrep)
soup = BeautifulSoup(html.text)

testdick = {' '.join([abbr[tst.findAll('strong')[0].text[-3:-1]],tst.findAll('strong')[2].text.split()[-1].lower()]) : tst.h5.a['href'][10:] for tst in soup.findAll('div', class_='span-4 last')[:len(df)]}
df['link'] = df['District'].map(testdick)
linkdict = {df['Member'].values[x]:df['link'].values[x] for x in range(len(df))}

# finds and assigns gender
alphalist = df['Member'].values
femwiki = 'https://en.wikipedia.org/wiki/Women_in_the_United_States_House_of_Representatives'
femhtml = requests.get(femwiki)
chickensoup = BeautifulSoup(femhtml.text)
femtable = chickensoup.findAll('table',class_='wikitable sortable')[-1]
femlist = [line.findChildren('td')[1].text.split('(')[0] for line in femtable.find('tbody').findAll('tr')[1:]]
gender = {x:0 if x in femlist else 1 for x in alphalist}
df['gender'] = df['Member'].map(gender)

In [ ]:
# Crawls and snaps floor speeches

urd = 'https://justfacts.votesmart.org/candidate/public-statements/170172/jack-bergman?speechType=14'

hdict = list(linkdict.keys())
speeches = {}
for x in hdict:
    speeches[x] = []
for x in range(len(hdict)):
    track = requests.get(url+linkdict[hdict[x]]+'?speechType=14')
    borsch = BeautifulSoup(track.text)
    for link in borsch.findAll('td',{'class':'statements-table-data'}):
        te = link.find('a')['href']
        kharcho = BeautifulSoup(requests.get(te).text)
        try:
            speeches[hdict[x]].append(kharcho.find('div',{'id':"publicStatementDetailSpeechContent"}).text)
        except:
            speeches[hdict[x]].append('')
    print('{}'.format((x+1)/len(df)))

In [198]:
job = speeches

In [202]:
bit = 'BREAK IN TRANSCRIPT'

# #first clipping -- ONLY RUN ONCE
# for y in range(len(hdict)):
#     for x in range(len(job[hdict[y]])):
#         test = job[hdict[y]][x]
#         job[hdict[y]][x] = test[test.find(bit)+len(bit):test.rfind(bit)]

# # fully cleaned texts
# cleaned = {}
# for y in range(len(hdict)):
#     for x in range(len((job[hdict[y]]))):
#         job[hdict[y]][x] = job[hdict[y]][x].split(bit)
#     testlist = [item for sublist in job[hdict[y]] for item in sublist]
#     cleaned[hdict[y]] = [item for item in testlist if item != '']

jobject = json.dumps(cleaned)
with open('clean_house_2021.json','x') as file:
    file.write(jobject)

In [203]:
# makes dictionary from csv/txt
def webster(file, type=1):
    if type == 1:
        dic = dict(csv.reader(open(file, 'r')))
    else:
        dic = dict([line.split() for line in open(file).read().replace('\t', ' ').split('\n') if len(line) != 0])
    for key in dic:
        dic[key] = float(dic[key])
    return dic

# subsection - dictionaries
valence = webster('valence.csv', type=1)
arousal = webster('arousal.csv', type=1)
dominance = webster('dominance.csv', type=1)
anger = webster('anger-scores.txt', type=0)
sadness = webster('sadness-scores.txt', type=0)
anticipation = webster('anticipation-scores.txt', type=0)
disgust = webster('disgust-scores.txt', type=0)
fear = webster('fear-scores.txt', type=0)
joy = webster('joy-scores.txt', type=0)
surprise = webster('surprise-scores.txt', type=0)
trust = webster('trust-scores.txt', type=0)
humor = pd.read_csv('humor_dataset.csv')
humor_tot = dict(zip(humor['word'], humor['mean']))
humor_M = dict(zip(humor['word'], humor['mean_M']))
humor_F = dict(zip(humor['word'], humor['mean_F']))
humor_young = dict(zip(humor['word'], humor['mean_young']))
humor_old = dict(zip(humor['word'], humor['mean_old']))
selfish = ['I’m', "i'll", 'i’ll', 'me', 'I’ll', 'i’d', "i'd", 'my', 'myself', 'i’m', "i'm", 'i’ve', "i've", 'I', 'i’d’ve', "i'd've", 'i', 'mine']

# create local / global averages
def emoav(emo, selfemo, wrdct):
    emo_len = len(selfemo)
    perc_emo = emo_len / wrdct
    scores = [emo[word] for word in selfemo]
    if len(scores) == 0:
      return 0, 0
    else:
      avg_loc = np.mean(scores)
      avg_glo = avg_loc * perc_emo
      return avg_loc, avg_glo

class Stripper:
    """For getting basic info from a file"""
    def __init__(self, text, person, age, gender):
        # imported info from excel
        self.stripped = text
        self.person = person
        self.age = age
        self.gender = gender       

        # top-line features
        sss = nltk.tokenize.sent_tokenize(self.stripped)
        self.sents = [x for x in sss if x[0].isalpha()]
        self.pos_sents = [nltk.pos_tag(self.sents[i].split(' ')) for i in range(len(self.sents)) if '-----------------' not in self.sents[i]]
        self.num_sents = len(self.sents)
        self.words_tok = nltk.tokenize.word_tokenize(self.stripped)
        self.words_raw = [x for x in self.stripped.split(' ')]
        self.wordcount = len(self.words_raw)
        self.clean_words = [word.lower().replace('.', '').replace(',', '') for word in self.words_raw]
        self.diff_words = set(self.clean_words)
        
        # POS features
        self.pos = nltk.pos_tag(self.words_tok)
        self.counts = Counter(tag for word,tag in self.pos)
        r = 0
        for key in self.counts:
            r += self.counts[key]
        r = r - (self.counts[','] + self.counts['.'])
        # all individual counts are relative to *pos counts* (not wordcounts)
        self.noun = 100 * (self.counts['NN'] + self.counts['NNS'] + self.counts['NNP'] + self.counts['NNPS']) / r
        self.pron = 100 * (self.counts['PRP'] + self.counts['PRP$']) / r
        self.adj = 100 * (self.counts['JJ'] + self.counts['JJR'] + self.counts['JJS']) / r
        self.adv = 100 * (self.counts['RB'] + self.counts['RBR'] + self.counts['RBS']) / r
        self.intj = 100 * (self.counts['UH']) / r
        self.verb = 100 * (self.counts['VB'] + self.counts['VBD'] + self.counts['VBG'] + self.counts['VBN'] + self.counts['VBP'] + self.counts['VBZ']) / r
        self.wh = 100 * (self.counts['WDT'] + self.counts['WP'] + self.counts['WP$'] + self.counts['WRB']) / r
        self.conj = 100 * (self.counts['CC']) / r
        self.prep = 100 * (self.counts['IN'] + self.counts['TO']) / r
        
        # fragments
        self.frag = 0
        self.sent_counts = [Counter(tag for word, tag in sent) for sent in self.pos_sents]
        verbs = {'VB','VBD','VBG','VBN','VBP','VBZ'}
        for sent in self.sent_counts:
            if len(verbs-set(sent.keys())) == len(verbs):
                self.frag += 1
                
        # word frequencies
        self.freq = nltk.FreqDist(word for word in self.clean_words)
        func = open('funcwords.txt').read().translate(str.maketrans("',[]", '    ')).split()
        self.funcs_freq = 100 * sum(self.freq[word] for word in self.freq if word in func) / self.wordcount
        self.rare = [word for word in self.freq if self.freq[word] == 1]
        self.rare_nonfunc = [word for word in self.rare if word not in func]
        self.selfish = sum(self.freq[word] for word in self.freq if word in selfish) / self.num_sents
        
        self.anger = [word for word in self.clean_words if word in anger.keys()]
        self.sadness = [word for word in self.clean_words if word in sadness.keys()]
        self.anticipation = [word for word in self.clean_words if word in anticipation.keys()]
        self.disgust = [word for word in self.clean_words if word in disgust.keys()]
        self.fear = [word for word in self.clean_words if word in fear.keys()]
        self.joy = [word for word in self.clean_words if word in joy.keys()]
        self.surprise = [word for word in self.clean_words if word in surprise.keys()]
        self.trust = [word for word in self.clean_words if word in trust.keys()]
        
        self.valence = [word for word in self.clean_words if word in valence.keys()]
        self.arousal = [word for word in self.clean_words if word in arousal.keys()]
        self.dominance = [word for word in self.clean_words if word in dominance.keys()]

        self.humor = [word for word in self.clean_words if word in humor_tot.keys()]
        
        # emotional strength
        self.al_fear, self.ag_fear = emoav(fear, self.fear, self.wordcount)
        self.al_joy, self.ag_joy = emoav(joy, self.joy, self.wordcount)
        self.al_trust, self.ag_trust = emoav(trust, self.trust, self.wordcount)
        self.al_surprise, self.ag_surprise = emoav(surprise, self.surprise, self.wordcount)
        self.al_disgust, self.ag_disgust = emoav(disgust, self.disgust, self.wordcount)
        self.al_anticipation, self.ag_anticipation = emoav(anticipation, self.anticipation, self.wordcount)
        self.al_anger, self.ag_anger = emoav(anger, self.anger, self.wordcount)
        self.al_sadness, self.ag_sadness = emoav(sadness, self.sadness, self.wordcount)

        self.al_valence, self.ag_valence = emoav(valence, self.valence, self.wordcount)
        self.al_arousal, self.ag_arousal = emoav(arousal, self.arousal, self.wordcount)
        self.al_dominance, self.ag_dominance = emoav(dominance, self.dominance, self.wordcount)

        self.al_humor_tot, self.ag_humor_tot = emoav(humor_tot, self.humor, self.wordcount)
        self.al_humor_M, self.ag_humor_M = emoav(humor_M, self.humor, self.wordcount)
        self.al_humor_F, self.ag_humor_F = emoav(humor_F, self.humor, self.wordcount)
        self.al_humor_young, self.ag_humor_young = emoav(humor_young, self.humor, self.wordcount)
        self.al_humor_old, self.ag_humor_old = emoav(humor_old, self.humor, self.wordcount)

        # sentence and word size
        self.short_sent, self.long_sent = 0, 0
        for sent in self.sents:
            if len(sent.split(' ')) <= 6:
                self.short_sent += 1
            else:
                self.long_sent += 1

        self.short_word, self.long_word = 0, 0
        for word in self.clean_words:
            if len(word) <= 6:
                self.short_word += 1
            else:
                self.long_word += 1
    
    # returns numpy array
    def bare(self):
        return np.array([self.person, self.age, self.gender, self.num_sents, self.wordcount, self.noun, self.verb, self.adj, self.adv, self.intj, self.pron, self.wh, self.conj, self.prep, int(self.short_sent), int(self.long_sent), self.short_word, self.long_word, self.frag, len(self.diff_words), self.funcs_freq, self.selfish, len(self.rare), (100*len(self.rare_nonfunc)/self.wordcount), (100*len(self.joy)/self.wordcount), (100*len(self.surprise)/self.wordcount), (100*len(self.anticipation)/self.wordcount), (100*len(self.anger)/self.wordcount), (100*len(self.fear)/self.wordcount), (100*len(self.trust)/self.wordcount), (100*len(self.disgust)/self.wordcount), (100*len(self.sadness)/self.wordcount), self.al_fear, self.ag_fear, self.al_joy, self.ag_joy, self.al_trust, self.ag_trust, self.al_surprise, self.ag_surprise, self.al_disgust, self.ag_disgust, self.al_anticipation, self.ag_anticipation, self.al_anger, self.ag_anger, self.al_sadness, self.ag_sadness, self.al_valence, self.ag_valence, self.al_arousal, self.ag_arousal, self.al_dominance, self.ag_dominance, self.al_humor_tot, self.ag_humor_tot, self.al_humor_M, self.ag_humor_M, self.al_humor_F, self.ag_humor_F, self.al_humor_young, self.ag_humor_young, self.al_humor_old, self.ag_humor_old, (100*len(self.humor)/self.wordcount), (100*len(self.valence)/self.wordcount)])
        
# converts arrays into dataframe        
def cashout(arrays):
    temp = [[arrays[i].bare()] for i in range(len(arrays))]
    s = pd.DataFrame(np.concatenate(temp), columns=['person', 'age', 'gender', 'sents','wordcount','noun','verb','adj','adv','intj','pron','wh','conj','prep','short sent','long sent','short word','long word','frag','# diff','% func words', 'selfish', '# unique', '% unique no func', '% joy', '% surprise', '% anticipation', '% anger', '% fear', '% trust', '% disgust', '% sadness', 'loc fear', 'glb fear', 'loc joy', 'glb joy', 'loc trust', 'glb trust', 'loc surprise', 'glb surprise', 'loc disgust', 'glb disgust', 'loc anticipation', 'glb anticipation', 'loc anger', 'glb anger', 'loc sadness', 'glb sadness', 'loc valence', 'glb valence', 'loc arousal', 'glb arousal', 'loc dominance', 'glb dominance', 'loc humor', 'glb humor', 'loc humor M', 'glb humor M', 'loc humor F', 'glb humor F', 'loc humor young', 'glb humor young', 'loc humor old', 'glb humor old', '% humor', '% VAD'])

    # type conversions
    stringcol = ['person']
    intcol = ['age', 'gender', 'sents', 'wordcount', 'short sent', 'long sent','short word', 'long word', 'frag', '# diff']
    floatcol = ['noun', 'verb', 'adj', 'adv', 'intj', 'pron', 'wh', 'conj', 'prep','% func words', 'selfish','# unique', '% unique no func', '% joy', '% surprise', '% anticipation','% anger', '% fear', '% trust', '% disgust', '% sadness', 'loc fear','glb fear', 'loc joy', 'glb joy', 'loc trust', 'glb trust','loc surprise', 'glb surprise', 'loc disgust', 'glb disgust','loc anticipation', 'glb anticipation', 'loc anger', 'glb anger','loc sadness', 'glb sadness', 'loc valence', 'glb valence','loc arousal', 'glb arousal', 'loc dominance', 'glb dominance','loc humor', 'glb humor', 'loc humor M', 'glb humor M', 'loc humor F','glb humor F', 'loc humor young', 'glb humor young', 'loc humor old','glb humor old', '% humor', '% VAD'] 
    s[stringcol] = s[stringcol].astype(str)
    s[intcol] = s[intcol].astype(str).astype(int)
    s[floatcol] = s[floatcol].astype(str).astype(float)
    return s

# calculates composite measures into the dataframe
def dance(ndf):
    ndf['short/long'] = ndf['short sent'] / ndf['long sent']
    ndf['words/sent'] = ndf['wordcount'] / ndf['sents']
    ndf['conj/sent'] = ndf['conj'] / ndf['sents']
    ndf['verb/noun'] = ndf['verb'] / ndf['noun']
    ndf['noun/pron'] = ndf['noun'] / ndf['pron']
    ndf['% diff w/func'] = ndf['# diff'] / ndf['wordcount'] * 100
    ndf['% unique w/func'] = ndf['# unique'] / ndf['wordcount'] * 100
    ndf['misc words'] = ndf['wordcount'] - (ndf['noun'] + ndf['verb'] + ndf['adj'] + ndf['adv'] + ndf['pron'] + ndf['intj'] + ndf['wh'] + ndf['prep'] + ndf['conj'])
    ndf['% frag'] = ndf['frag'] / ndf['sents'] * 100
    
# does all of the above in one line with the option of switching to manipulate a single object
def VIP(dic):
    temp = cashout(dic)
    dance(temp)
    return temp
    
def tablemaker(atts,name,alsoticks='False',time='False'):
    if time == 'False':
        bx = biden[atts].mean()
        tx = trump[atts].mean()
        bxerr = biden[atts].std()/sqrt(len(biden.index))
        txerr = trump[atts].std()/sqrt(len(trump.index))
    else:
        bx = biden_timed[atts].mean()
        tx = trump_timed[atts].mean()
        bxerr = biden_timed[atts].std()/sqrt(len(biden_timed.index))
        txerr = trump_timed[atts].std()/sqrt(len(trump_timed.index))
        

    Z = np.arange(len(atts))
    plt.bar(Z-.1, bx, color='#0015BC', width = .2, label='Biden', yerr=bxerr)
    plt.bar(Z+.1, tx, color='#DE0100', width = .2, label='Trump', yerr=txerr)

    plt.ylabel('Score')
    plt.title(name)
    if alsoticks=='False':
        plt.xticks(Z, tuple(atts))
    else:
        plt.xticks(Z, tuple(alsoticks))
    plt.legend(loc='best')
    plt.figure(figsize=(30,20))
#     plt.savefig(name+'.png')
    plt.show()

In [204]:
with open('clean_house_2021.json','r') as openfile:
    cleanjob = json.load(openfile)

for y in range(len(hdict)):
    for x in range(len(cleanjob[hdict[y]])):
        test = cleanjob[hdict[y]][x]
        cleanjob[hdict[y]][x] = test.replace('\n',' ').replace('\n ',' ').replace('  \n',' ').replace('   ',' ').replace('    ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').strip()
    cleanjob[hdict[y]] = [u for u in cleanjob[hdict[y]] if len(u.split(' ')) > 20]
    cleanjob[hdict[y]] = [u for u in cleanjob[hdict[y]] if 'M' in u[:2]]

In [ ]:
allreps = []
for y in range(len(hdict)):
    print('y-{}--rep-{}'.format(1+y,hdict[y]))
    for x in range(len(cleanjob[hdict[y]])):
        allreps.append(Stripper(cleanjob[hdict[y]][x], hdict[y], int(df[df['Member']==hdict[y]]['Age']), int(df[df['Member']==hdict[y]]['gender'])))
        print('{} of {}'.format(1+x,len(cleanjob[hdict[y]])))
    

In [207]:
ALLREPS = VIP(allreps)
ALLREPS.to_pickle('allreps_2021.pkl')

In [ ]:
tot = 0
for x in hdict:
    print('{}: {} texts'.format(x,len(cleanjob[x])))
    tot += len(cleanjob[x])
print(tot, tot/len(df))

In [213]:
df.to_csv('house2021.csv')